In [3]:
from pynq import Overlay
from pynq import MMIO
from pynq import allocate
import numpy as np
import math
import os


# Program bitstream to FPGA
overlay = Overlay('./DES_axi_wrapper.bit')

# Access to AXI DMA
dma = overlay.axi_dma_0
dma_send = dma.sendchannel
dma_recv = dma.recvchannel

/home/xilinx/jupyter_notebooks/thies/notebooks/DMA


In [4]:

SIZE_OF_ALGO = 8 # 8 bytes = 64 bits
MAX_PARALLEL_SEND = 8

input_type = np.dtype([("message", np.bytes_, SIZE_OF_ALGO), ("key", np.bytes_, SIZE_OF_ALGO)])
output_type = np.dtype([("result", np.bytes_, SIZE_OF_ALGO)])
print(f"Using {MAX_PARALLEL_SEND}x type with input: {input_type.itemsize * 8} bits, and output: {output_type.itemsize * 8} bits")

# Allocate physical memory
input_buffer = allocate(shape=(MAX_PARALLEL_SEND,), dtype=input_type)
output_buffer = allocate(shape=(MAX_PARALLEL_SEND,), dtype=output_type)

Using 8x type with input: 128 bits, and output: 64 bits


In [5]:
message = "AAAAAAAA"
key = "BBBBBBBB"

key_bytes = bytes(key, "ascii")
assert(len(key_bytes) == 8), len(key_bytes)
print(key_bytes.hex())

message_bytes = bytes(message, "ascii")
assert(len(message_bytes) == 8), len(key_bytes)
print(message_bytes.hex())

iterations = math.ceil(len(message_bytes) / SIZE_OF_ALGO)
assert(iterations < MAX_PARALLEL_SEND)

4242424242424242
4141414141414141


In [6]:
# Write input
for i in range(iterations):
    part = message_bytes[8 * i : 8 * (i + 1)]
    input_buffer[i] = (message, key)
    
    print(input_buffer[i])

(b'AAAAAAAA', b'BBBBBBBB')


In [7]:
# Do AXI DMA MM2S transfer
dma_send.transfer(input_buffer)
# Do AXI DMA S2MM transfer
dma_recv.transfer(output_buffer)

In [9]:
# Check the memory content after DMA transfer
for i in range(iterations):
    print(output_buffer[i].tobytes()[::-1].hex())
    
assert output_buffer[i].tobytes()[::-1].hex() == "57FA2EC1BB39CA18".lower()

57fa2ec1bb39ca18


In [10]:

# Delete buffer to prevent memory leak
del input_buffer, output_buffer